# DIET PROBLEM - PYOMO

*Zuria Bauer Hartwig and David Senén García Hurtado* ( [CAChemE](http://cacheme.org))

Sources:

* Original Problem: [Linear and Integer Programming](https://www.coursera.org/course/linearprogramming) (Coursera Course) - University of Colorado Boulder & University of Colorado System

* Based on the Examples from the Optimization Course = [Taller-Optimizacion-Python-Pyomo](https://github.com/CAChemE/Taller-Optimizacion-Python-Pyomo) from [CAChemE.org](http://cacheme.org/optimizacion-programacion-matematica-con-python-pyomo/)

* [Neos Guide - The Diet Problem](http://www.neos-guide.org/content/diet-problem)

## Big picture

Now let's play with Pyomo. What if we want to maximize the amount of calories we can get for less than $5?

<img src="img/now-talking.jpg" alt="talking" style="width: 500px;"/>

#### 1. Import

In [3]:
from pyomo.environ import *
infinity = float('inf')

# We are importing Pyomo library to our Python enviroment

# Side-note:
# Usually importing all (*) the library is a VERY BAD idea
# By convention Pyomo documentation does that so here we are

# 2. Data

We can use the following data:

- http://nutrition.mcdonalds.com/getnutrition/nutritionfacts.pdf (dead link)
- http://fastfoodnutrition.org/mcdonalds/chart
- http://www.fastfoodmenuprices.com/mcdonalds-prices/

In [1]:
# Let's have a look to the data:
# Mac or Linux ('!' meaning to be run through the terminal)
!cat data/foodmax.dat

# Windows via cmd.exe
#!type data\foodmax.dat

# http://nutrition.mcdonalds.com/getnutrition/nutritionfacts.pdf
# http://www.fastfoodmenuprices.com/mcdonalds-prices/

#Food + Cost

param:  F:                                                 Cal  :=
  "Bacon Clubhouse Burgerx"                                740
  "Bacon Clubhouse Grilled Chicken Sandwichx"              610
  "Bacon Clubhouse Crispy Chicken Sandwichx"               750             
  "McChickenx"                                             370
  "Filet-O-Fishx"                                          470             
  "McRibx"                                                 390       
  "Premium McWrap Chicken & Bacon (Crispy)x"               500
  "Premium McWrap Chicken & Bacon (Grilled)x"              640
  "Premium McWrap Chicken & Ranch (Crispy)x"               500             
  "Premium McWrap Chicken & Ranch (Grilled)x"              610     
  "Premium McWrap Chicken & Sweet Chili (Crispy)x"         470
  "Premium McWrap Chicken & Sweet Chili (Grilled)x"    

**Note**: Window's users can use `!type data\food-max.dat` command instead of `!cat data/food-max.dat`

### Solving

#### 3. Model

In [5]:
# The AbstractModel class provides a context for defining and initializing abstract optimization models in Pyomo

model = AbstractModel()

#### 4. Sets

In [6]:
## DEFINE SETS
# Set data that is used to define a model instance

# Products
model.F = Set()
# Nutrients
model.N = Set()

#### 5. Parameters

In [7]:
## DEFINE PARAMETERS
# Parameter data that is used to define a model instance

# Calories
model.Cal = Param(model.F, within = PositiveReals)

# Cost
model.a    = Param(model.F, model.N, within = NonNegativeReals)

# Max and Min Cost
model.Cmin = Param(model.N, within = NonNegativeReals, default = 0.0)
model.Cmax = Param(model.N, within = NonNegativeReals, default = infinity)

#### 6. Variables

In [8]:
## VARIABLES
# Decision variables in a model

# Number of servings
model.x = Var(model.F, within = NonNegativeIntegers)


#### 7. Objective

In [9]:
## OBJECTIVE
# Expressions that are minimized or maximized in a model

# MAXIMIZE z(calories)
def calories(model):
    return sum(model.Cal[i] * model.x[i] for i in model.F)
model.calories = Objective(rule=calories, sense=maximize)

#### 8. Constrains

In [10]:
## CONSTRAINS
# Constraint expressions that impose restrictions on variable values in a model

#Max
def cost_max(model, j):
    value = sum(model.a[i,j] * model.x[i] for i in model.F)
    return value <= model.Cmax[j]
model.cost_limit_max = Constraint(model.N, rule=cost_max)

#Min
def cost_min(model, j):
    value = sum(model.a[i,j] * model.x[i] for i in model.F)
    return model.Cmin[j] <= value 
model.cost_limit_min = Constraint(model.N, rule=cost_min)

#### 9. Solution

In [15]:
##SOLUTION
#Get our Solution:

!pyomo solve --solver=glpk problems/diet-maximize/pyomo-max.py data/food-max.dat

# windows
#!pyomo solve --solver=glpk problems\diet-maximize\pyomo-max.py data\food-max.dat

[    0.00] Setting up Pyomo environment
[    0.00] Applying Pyomo preprocessing actions
[    0.00] Creating model
[    0.03] Applying solver
[    0.04] Processing results
    Number of solutions: 1
    Solution Information
      Gap: 0.0
      Status: optimal
      Function Value: 2690.0
    Solver results file: results.yml
[    0.04] Applying Pyomo postprocessing actions
[    0.04] Pyomo Finished


#### 10. Results

In [16]:
## Lets have a look to the results now since we are hungry!

# Mac or Linux ('!' meaning to be run through the terminal)
!cat results.yml

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 2690.0
  Upper bound: 2690.0
  Number of objectives: 1
  Number of constraints: 3
  Number of variables: 39
  Number of nonzeros: 77
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 3
      Number of created subproblems: 3
  Error rc: 0
  Time: 0.0051555633544921875
# ----------------------------------------------------------
#   Solution Information
# -------------------------------

##### Results

- ** 3 plain hamburguers =** 1.29€
<img src="img/Hamburger.jpg" alt="Hamburger" style="width: 300px;"/>

- ** 1 Grilled Onion Rings =** 1.00€
<img src="img/onion-rings.png" alt="Onion" style="width: 300px;"/>